# Init

In [ ]:
import sys
sys.path.append(r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\simcore-sdk\src")
sys.path.append(r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\s3wrapper\src")

import logging
logging.basicConfig(level=logging.WARNING)

In [ ]:
import os
os.environ["SIMCORE_NODE_UUID"] = "12345555"
os.environ["PIPELINE_NODE_ID"] = "1"

os.environ["POSTGRES_ENDPOINT"] = "localhost:5432"
os.environ["POSTGRES_USER"] = "simcore"
os.environ["POSTGRES_PASSWORD"] = "simcore"
os.environ["POSTGRES_DB"] = "simcoredb"

os.environ["S3_ENDPOINT"] = "localhost:9001"
os.environ["S3_ACCESS_KEY"] = "12345678"
os.environ["S3_SECRET_KEY"] = "12345678"
os.environ["S3_BUCKET_NAME"] = "simcore"


# Use of nodeports

In [ ]:
from simcore_sdk.nodeports import PORTS

In [ ]:
print(len(PORTS.inputs))
print(PORTS.inputs[0])

In [ ]:
print(PORTS.inputs[0].get())


In [ ]:
print(PORTS.outputs[0])

In [ ]:
PORTS.outputs["out_1"].get()

In [ ]:
PORTS.outputs["out_1"].set("Hey this still works or NOTTTTTT crazy shit man!!")

# Set up database and S3 with some data from a configuration

In [ ]:
import os
import sys
import tempfile
import json
import pandas as pd
import numpy as np
import tenacity
from pathlib import Path

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from simcore_sdk.models.pipeline_models import Base, ComputationalTask, ComputationalPipeline
from simcore_sdk.config.db import Config as db_config

from simcore_sdk.config.s3 import Config as s3_config
from s3wrapper.s3_client import S3Client


class DbSettings(object):
    def __init__(self):
        self._db_config = db_config()
        self.db = create_engine(self._db_config.endpoint, client_encoding='utf8')
        self.Session = sessionmaker(self.db)
        self.session = self.Session()

class S3Settings(object):
    def __init__(self):
        self._config = s3_config()
        self.client = S3Client(endpoint=self._config.endpoint,
            access_key=self._config.access_key, secret_key=self._config.secret_key)
        self.bucket = self._config.bucket_name
        self.client.create_bucket(self.bucket)
@tenacity.retry(wait=tenacity.wait_fixed(2), stop=tenacity.stop_after_attempt(5) | tenacity.stop_after_delay(20))
def init_db():
    db = DbSettings()    
    Base.metadata.create_all(db.db)
    return db

@tenacity.retry(wait=tenacity.wait_fixed(2), stop=tenacity.stop_after_attempt(5) | tenacity.stop_after_delay(20))
def init_s3():
    s3 = S3Settings()
    return s3
db = init_db()
new_Pipeline = ComputationalPipeline()
db.session.add(new_Pipeline)

db.session.commit()

In [ ]:
node_uuid = os.environ.get("SIMCORE_NODE_UUID")
configuration={
    "version":"0.1",
    "inputs": [
        {
            "key": "in_1",
            "label": "vm 1Hz",
            "desc": "these are computed data out of a pipeline",
            "type": "fileUrl",
            "value": "link." + node_uuid + ".in_1",
            "timestamp": "2018-05-23T15:34:53.511Z"
        },
        {
            "key": "in_2",
            "label": "all results 1Hz",
            "desc": "computed data out of a pipeline",
            "type": "fileUrl",
            "value": "link." + node_uuid + ".in_2",
            "timestamp": "2018-05-23T15:34:53.511Z"
        }
    ],
    "outputs": [        
    ]
}

#configuration = json.loads(json_configuration)
new_Node = ComputationalTask(pipeline_id=new_Pipeline.pipeline_id, node_id=node_uuid, input=configuration["inputs"], output=configuration["outputs"])
db.session.add(new_Node)
db.session.commit()

In [ ]:
def create_dummy_table(number_of_rows, number_of_columns):
    time = np.arange(number_of_rows).reshape(number_of_rows,1)
    matrix = np.random.randn(number_of_rows, number_of_columns)
    fullmatrix = np.hstack((time, matrix))
    df = pd.DataFrame(fullmatrix)
    return df

# create a dummy file filled with dummy data
temp_file = tempfile.NamedTemporaryFile()
temp_file.close()

# create a dummy table
number_of_rows = 5000
number_of_columns = 200
number_of_files = 20
s3 = init_s3()
# push the file to the S3 for each input item
for input_item in configuration["inputs"]:
    if input_item["type"] == "fileUrl":
        df = create_dummy_table(number_of_rows, number_of_columns)
        # serialize to the file
        with open(temp_file.name, "w") as file_pointer:
            df.to_csv(path_or_buf=file_pointer, sep="\t", header=False, index=False)        

        s3_object_name = Path(str(new_Pipeline.pipeline_id), node_uuid, input_item["key"])
        s3.client.upload_file(s3.bucket, s3_object_name.as_posix(), temp_file.name)
    elif input_item["type"] == "folderUrl":
        for i in range(number_of_files):
            df = create_dummy_table(number_of_rows, number_of_columns)
            # serialize to the file
            with open(temp_file.name, "w") as file_pointer:
                df.to_csv(path_or_buf=file_pointer, sep="\t", header=False, index=False)        

            s3_object_name = Path(str(new_Pipeline.pipeline_id), node_uuid, input_item["key"], str(i) + ".dat")
            s3.client.upload_file(s3.bucket, s3_object_name.as_posix(), temp_file.name)

Path(temp_file.name).unlink()